# Experiment tracking

**Important:** `SQliteTracker` got a big upgrade in version `0.8.2` so ensure you are running such version or a higher one.

`SQLiteTracker` provides a powerful and flexible way to track computational (e.g., Machine Learning) experiments using a SQLite database. Allows you to use SQL as the query language, giving you a powerful tool for experiment comparison, and comes with plotting features to compare plots side-by-side and to combine plots for better comparison.

Read more about the motivations in our [blog post](https://ploomber.io/blog/experiment-tracking/), check out the [HN discussion.](https://news.ycombinator.com/item?id=33624018)

This tutorial will walk you through the features with a Machine Learning use case; however, the tracker is generic enough to be used in any other domains.

In [1]:
from pathlib import Path

import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, RocCurveDisplay

In [2]:
# delete our example database, if any
db = Path('my_experiments.db')

if db.exists():
    db.unlink()

In [3]:
from sklearn_evaluation import SQLiteTracker

tracker = SQLiteTracker('my_experiments.db')

In [4]:
X, y = datasets.make_classification(200, 10, n_informative=5, class_sep=0.65)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

models = [RandomForestClassifier(), LogisticRegression(), DecisionTreeClassifier()]

## Training and logging models

In [5]:
for m in models:
    model = type(m).__name__
    print(f'Fitting {model}')


    experiment = tracker.new_experiment()
    m.fit(X_train, y_train)
    y_pred = m.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    
    # log a dictionary with log_dict
    experiment.log_dict({'accuracy': acc, 'model': model, **m.get_params()})

Fitting RandomForestClassifier
Fitting LogisticRegression
Fitting DecisionTreeClassifier


Or use `.log(key, value)` to log individual values:

In [6]:
svc = SVC()
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
acc = accuracy_score(y_test, y_pred)

experiment = tracker.new_experiment()

# log individual values
experiment.log('accuracy', acc)
experiment.log('model', type(svc).__name__)

_ = experiment.log_dict(svc.get_params())

## Displaying latest experiments

Display the `tracker` object to show last experiments:

In [7]:
tracker

uuid,created,parameters,comment
e5881cd9,2022-12-20 10:27:26,"{""accuracy"": 0.8484848484848485, ""model"": ""SVC"", ""C"": 1.0, ""break_ties"": false, ""cache_size"": 200, ""class_weight"": null, ""coef0"": 0.0, ""decision_function_shape"": ""ovr"", ""degree"": 3, ""gamma"": ""scale"", ""kernel"": ""rbf"", ""max_iter"": -1, ""probability"": false, ""random_state"": null, ""shrinking"": true, ""tol"": 0.001, ""verbose"": false}",
00dc54e6,2022-12-20 10:27:13,"{""accuracy"": 0.803030303030303, ""model"": ""LogisticRegression"", ""C"": 1.0, ""class_weight"": null, ""dual"": false, ""fit_intercept"": true, ""intercept_scaling"": 1, ""l1_ratio"": null, ""max_iter"": 100, ""multi_class"": ""auto"", ""n_jobs"": null, ""penalty"": ""l2"", ""random_state"": null, ""solver"": ""lbfgs"", ""tol"": 0.0001, ""verbose"": 0, ""warm_start"": false}",
65112cd8,2022-12-20 10:27:13,"{""accuracy"": 0.7727272727272727, ""model"": ""DecisionTreeClassifier"", ""ccp_alpha"": 0.0, ""class_weight"": null, ""criterion"": ""gini"", ""max_depth"": null, ""max_features"": null, ""max_leaf_nodes"": null, ""min_impurity_decrease"": 0.0, ""min_samples_leaf"": 1, ""min_samples_split"": 2, ""min_weight_fraction_leaf"": 0.0, ""random_state"": null, ""splitter"": ""best""}",
fc1df950,2022-12-20 10:27:12,"{""accuracy"": 0.8333333333333334, ""model"": ""RandomForestClassifier"", ""bootstrap"": true, ""ccp_alpha"": 0.0, ""class_weight"": null, ""criterion"": ""gini"", ""max_depth"": null, ""max_features"": ""sqrt"", ""max_leaf_nodes"": null, ""max_samples"": null, ""min_impurity_decrease"": 0.0, ""min_samples_leaf"": 1, ""min_samples_split"": 2, ""min_weight_fraction_leaf"": 0.0, ""n_estimators"": 100, ""n_jobs"": null, ""oob_score"": false, ""random_state"": null, ""verbose"": 0, ""warm_start"": false}",


## Querying experiments with SQL using `.query()`

You can use SQL to query your experiments. To see what's been logged, use `get_parameters_keys()`:

In [8]:
keys = tracker.get_parameters_keys()
# show first 5 keys
keys[:5]

['C', 'accuracy', 'bootstrap', 'break_ties', 'cache_size']

To generate a sample query, use `.get_sample_query()`:

In [9]:
print(tracker.get_sample_query())

SELECT
    uuid,
    json_extract(parameters, '$.C') as C,
    json_extract(parameters, '$.accuracy') as accuracy,
    json_extract(parameters, '$.bootstrap') as bootstrap,
    json_extract(parameters, '$.break_ties') as break_ties,
    json_extract(parameters, '$.cache_size') as cache_size,
    json_extract(parameters, '$.ccp_alpha') as ccp_alpha,
    json_extract(parameters, '$.class_weight') as class_weight,
    json_extract(parameters, '$.coef0') as coef0,
    json_extract(parameters, '$.criterion') as criterion,
    json_extract(parameters, '$.decision_function_shape') as decision_function_shape,
    json_extract(parameters, '$.degree') as degree,
    json_extract(parameters, '$.dual') as dual,
    json_extract(parameters, '$.fit_intercept') as fit_intercept,
    json_extract(parameters, '$.gamma') as gamma,
    json_extract(parameters, '$.intercept_scaling') as intercept_scaling,
    json_extract(parameters, '$.kernel') as kernel,
    json_extract(parameters, '$.l1_ratio') as l1_

To execute a query, use `.query()`:

In [10]:
ordered = tracker.query("""
SELECT uuid,
       json_extract(parameters, '$.model') AS model,
       json_extract(parameters, '$.accuracy') AS accuracy
FROM experiments
ORDER BY accuracy DESC
""")
ordered

,model,accuracy
uuid,,
e5881cd9,SVC,0.848485
fc1df950,RandomForestClassifier,0.833333
00dc54e6,LogisticRegression,0.803030
65112cd8,DecisionTreeClassifier,0.772727


The query method returns a data frame with "uuid" as the index:

In [11]:
type(ordered)

pandas.core.frame.DataFrame

## Storing plots

You can log a confusion matrix and classification reports:

In [12]:
%%capture

def fit(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    experiment = tracker.new_experiment()
    experiment.log_dict({'accuracy': acc, 'model': type(model).__name__, **model.get_params()})
    
    # log plots     
    experiment.log_confusion_matrix(y_test, y_pred)
    experiment.log_classification_report(y_test, y_pred)
    
    # log generic matplotlib figure
    roc = RocCurveDisplay.from_estimator(model, X_test, y_test)
    experiment.log_figure('roc', roc.figure_)
    
fit(model=RandomForestClassifier(n_estimators=100))
fit(model=RandomForestClassifier(n_estimators=10))

In [13]:
tracker.recent(2)

,created,parameters,comment
uuid,,,
563d64b4,2022-12-20 10:27:42,"{""accuracy"": 0.7575757575757576, ""model"": ""Ran...",None
ce575ef8,2022-12-20 10:27:41,"{""accuracy"": 0.803030303030303, ""model"": ""Rand...",None


## Rendering plots in table view

The `.query()` method also allows rendering plots in the table view:

In [14]:
results = tracker.query("""
SELECT uuid,
       json_extract(parameters, '$.model') AS model,
       json_extract(parameters, '$.accuracy') AS accuracy,
       json_extract(parameters, '$.confusion_matrix') AS cm,
       json_extract(parameters, '$.roc') AS roc
FROM experiments
ORDER BY created DESC
LIMIT 2
""", as_frame=False, render_plots=True)

results

uuid,model,accuracy,cm,roc
563d64b4,RandomForestClassifier,0.757576,,
ce575ef8,RandomForestClassifier,0.803030,,


## Side-by-side comparison

From the `.query()` results, you can extract a given column for a side by side comparison:

In [15]:
results.get("cm")

You can change the labels in the tabs with the `index_by` argument:

In [16]:
results.get("cm", index_by="accuracy")

## Combining plots

With a side-by-side comparison, it might be hard to spot the model performance differents, you can get individual experiments, extract their plots and combine them:

In [17]:
# get the uuids for the latest 2 experiments
uuid1, uuid2 = tracker.recent(2).index

# get the experiments
one = tracker.get(uuid1)
another = tracker.get(uuid2)

Combine statistics from both confusion matrices:

In [18]:
one["confusion_matrix"] + another["confusion_matrix"]

Get confusion matrix differences:

In [19]:
one["confusion_matrix"] - another["confusion_matrix"]

Combine classification reports:

In [20]:
one["classification_report"] + another["classification_report"]

You can also retrieve generic figures (logged with `log_figure`):

In [21]:
one["roc"]

However, note that plot combination (`plot1 + plot2` and `plot1 - plot2`) is only supported by plots logged via the `experiment.log_*` and not by the generic `experiment.log_figure` method.

## Adding comments

In [22]:
one.comment('This is some comment')

In [23]:
tracker.query("""
SELECT uuid,
       comment,
       json_extract(parameters, '$.model') AS model,
       json_extract(parameters, '$.accuracy') AS accuracy
FROM experiments
WHERE comment is not NULL
""")

,comment,model,accuracy
uuid,,,
563d64b4,This is some comment,RandomForestClassifier,0.757576


## Pandas integration

### Getting recent experiments

The recent method also returns a data frame:

In [24]:
df = tracker.recent()
df

,created,parameters,comment
uuid,,,
563d64b4,2022-12-20 10:27:42,"{""accuracy"": 0.7575757575757576, ""model"": ""Ran...",This is some comment
ce575ef8,2022-12-20 10:27:41,"{""accuracy"": 0.803030303030303, ""model"": ""Rand...",None
e5881cd9,2022-12-20 10:27:26,"{""accuracy"": 0.8484848484848485, ""model"": ""SVC...",None
00dc54e6,2022-12-20 10:27:13,"{""accuracy"": 0.803030303030303, ""model"": ""Logi...",None
65112cd8,2022-12-20 10:27:13,"{""accuracy"": 0.7727272727272727, ""model"": ""Dec...",None


Pass `normalize=True` to convert the nested JSON dictionary into columns:

In [27]:
df = tracker.recent(normalize=True)
df

,created,accuracy,model,bootstrap,ccp_alpha,class_weight,criterion,max_depth,max_features,max_leaf_nodes,...,tol,dual,fit_intercept,intercept_scaling,l1_ratio,multi_class,penalty,solver,splitter,comment
uuid,,,,,,,,,,,,,,,,,,,,,
563d64b4,2022-12-20 10:27:42,0.757576,RandomForestClassifier,True,0.0,None,gini,NaN,sqrt,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,This is some comment
ce575ef8,2022-12-20 10:27:41,0.803030,RandomForestClassifier,True,0.0,None,gini,NaN,sqrt,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
e5881cd9,2022-12-20 10:27:26,0.848485,SVC,NaN,NaN,None,NaN,NaN,NaN,NaN,...,0.0010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
00dc54e6,2022-12-20 10:27:13,0.803030,LogisticRegression,NaN,NaN,None,NaN,NaN,NaN,NaN,...,0.0001,False,True,1.0,NaN,auto,l2,lbfgs,NaN,None
65112cd8,2022-12-20 10:27:13,0.772727,DecisionTreeClassifier,NaN,0.0,None,gini,NaN,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,best,None


### Querying

You can also use the `.query()` method with `as_frame=True` (default value) to get a `pandas.DataFrame`

In [28]:
df = tracker.query("""
SELECT uuid,
       json_extract(parameters, '$.model') AS model,
       json_extract(parameters, '$.accuracy') AS accuracy
FROM experiments
ORDER BY accuracy DESC
LIMIT 3
""")
df

,model,accuracy
uuid,,
e5881cd9,SVC,0.848485
fc1df950,RandomForestClassifier,0.833333
00dc54e6,LogisticRegression,0.803030
